In [7]:
import random
import pandas as pd

In [8]:
mappings = {
  'San Francisco 49ers': '49ers',
  'Chicago Bears': 'Bears',
  'Cincinnati Bengals': 'Bengals',
  'Buffalo Bills': 'Bills',
  'Denver Broncos': 'Broncos',
  'Cleveland Browns': 'Browns',
  'Tampa Bay Buccaneers': 'Buccaneers',
  'Arizona Cardinals': 'Cardinals',
  'Los Angeles Chargers': 'Chargers',
  'Kansas City Chiefs': 'Chiefs',
  'Indianapolis Colts': 'Colts',
  'Washington Football Team': 'Commanders',
  'Washington Commanders': 'Commanders',
  'Dallas Cowboys': 'Cowboys',
  'Miami Dolphins': 'Dolphins',
  'Philadelphia Eagles': 'Eagles',
  'Atlanta Falcons': 'Falcons',
  'New York Giants': 'Giants',
  'Jacksonville Jaguars': 'Jaguars',
  'New York Jets': 'Jets',
  'Detroit Lions': 'Lions',
  'Green Bay Packers': 'Packers',
  'Carolina Panthers': 'Panthers',
  'New England Patriots': 'Patriots',
  'Las Vegas Raiders': 'Raiders',
  'Los Angeles Rams': 'Rams',
  'Baltimore Ravens': 'Ravens',
  'New Orleans Saints': 'Saints',
  'Seattle Seahawks': 'Seahawks', 
  'Pittsburgh Steelers': 'Steelers',
  'Houston Texans': 'Texans',
  'Tennessee Titans': 'Titans',
  'Minnesota Vikings': 'Vikings',
}

teams = list(set(mappings.values()))

In [18]:
def generate_team_power_ratings():
  return { team: random.random() for team in teams }

home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()

In [16]:
df = pd.read_csv('../../data/nfl/pfr-2021-games-combo.csv', index_col=0)

df = df[df['week'].str.isdigit()]

df['week'] = df['week'].astype(str)
df['home'] = df['home'].astype(str).map(lambda a: mappings[a])
df['home_pts'] = df['home_pts'].astype(int)
df['away'] = df['away'].astype(str).map(lambda a: mappings[a])
df['away_pts'] = df['away_pts'].astype(int)
df['margin'] = df.apply(lambda a: a['home_pts'] - a['away_pts'], axis=1)

In [42]:
def tweak_power_ratings(power_ratings):
   delta_power_ratings = power_ratings.copy()

   teams = list(delta_power_ratings.keys())

   k = random.randint(2, len(teams))
   for team in random.sample(teams, k):
      delta_power_ratings[team] += random.randint(-1, 1)
      delta_power_ratings[team] += random.random()

   return delta_power_ratings

tweak_power_ratings(home_power_ratings)

{'Cardinals': -5.811217383691646,
 'Texans': -15.665677968324147,
 'Bengals': -5.640096229206025,
 'Rams': 1.8712985360580952,
 'Seahawks': 1.0252026406895407,
 'Cowboys': 9.35356061602094,
 'Chargers': 1.5656890724105201,
 'Steelers': -0.24832761775037782,
 'Ravens': 0.8966224248037664,
 'Vikings': -1.109157255624897,
 'Lions': -6.340050128937503,
 'Saints': -4.506106594260404,
 'Bills': 1.3069007818663771,
 'Jaguars': -7.985008120404454,
 'Dolphins': -3.967118197366625,
 'Raiders': -8.50867509611511,
 'Jets': -11.793702809810322,
 'Commanders': -3.293617957841829,
 'Patriots': 6.745343785526179,
 'Falcons': -17.21015249435546,
 'Titans': 3.934149757496958,
 'Browns': -2.953802550311248,
 'Giants': -7.395686776859721,
 'Panthers': -11.418704923778165,
 'Buccaneers': 13.548403122052179,
 'Chiefs': 4.677005455320387,
 '49ers': 0.3999958534442781,
 'Eagles': -6.937321247493637,
 'Packers': 9.118078825374253,
 'Broncos': -1.9618026602998402,
 'Colts': -6.377039833253198,
 'Bears': -2.9196

In [20]:
def evaluate(df, home_power_ratings, away_power_ratings, home_edge = 3):
  def prediction(row):
    return home_edge + row['home_rating'] - row['away_rating']

  df_pred = df.copy()

  df_pred['home_rating'] = df_pred['home'].map(home_power_ratings)
  df_pred['away_rating'] = df_pred['away'].map(away_power_ratings)
  df_pred['prediction'] = df_pred.apply(prediction, axis=1)
  df_pred['error'] = df_pred.apply(lambda a: a['margin'] - a['prediction'], axis=1)
  df_pred['sq_error'] = df_pred['error'].map(lambda a: a * a)

  columns = [
    'week',
    'home',
    'home_pts',
    'home_rating',
    'away',
    'away_pts',
    'away_rating',
    'prediction',
    'error',
    'sq_error',
  ]

  return df_pred[columns]

df_pred = evaluate(df, home_power_ratings, away_power_ratings)

print('sq error:', df_pred['sq_error'].sum())
print()

df_pred.head()

sq error: 65040.9513407263



,week,home,home_pts,home_rating,away,away_pts,away_rating,prediction,error,sq_error
0,1,Buccaneers,31,0.811752,Cowboys,29,0.668118,3.143634,-1.143634,1.307900
1,1,Falcons,6,0.107527,Eagles,32,0.429924,2.677603,-28.677603,822.404923
2,1,Bills,16,0.170317,Steelers,23,0.257275,2.913041,-9.913041,98.268389
3,1,Panthers,19,0.333601,Jets,14,0.327549,3.006053,1.993947,3.975826
4,1,Bengals,27,0.824739,Vikings,24,0.591555,3.233183,-0.233183,0.054374


In [21]:
home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()
global_sq_error = evaluate(df, home_power_ratings, away_power_ratings)['sq_error'].sum()

In [51]:
print(f'global sq error @ 0:', global_sq_error)

debug = False
total_iterations_allowed_in_pass = 250
passes = 10

for i in range(1, passes):

  tgse = global_sq_error

  current_iteration = 1
  while current_iteration <= total_iterations_allowed_in_pass:
    delta_home_power_ratings = tweak_power_ratings(home_power_ratings)
    sq_error = evaluate(df, delta_home_power_ratings, away_power_ratings)['sq_error'].sum()
    if global_sq_error > sq_error:
      global_sq_error = sq_error
      home_power_ratings = delta_home_power_ratings

    delta_away_power_ratings = tweak_power_ratings(away_power_ratings)
    sq_error = evaluate(df, home_power_ratings, delta_away_power_ratings)['sq_error'].sum()
    if global_sq_error > sq_error:
      global_sq_error = sq_error
      away_power_ratings = delta_away_power_ratings

    current_iteration += 1

  if tgse != global_sq_error:
    print(f'global sq error @ {i}:', global_sq_error)
  


global sq error @ 0: 37500.07281248723
global sq error @ 1: 37499.809008792094
global sq error @ 2: 37499.346197912426
global sq error @ 8: 37499.324175371396
global sq error @ 9: 37499.025387269845


In [48]:
print('sq error:', global_sq_error)
evaluate(df, home_power_ratings, away_power_ratings).head()

sq error: 37500.07281248723


,week,home,home_pts,home_rating,away,away_pts,away_rating,prediction,error,sq_error
0,1,Buccaneers,31,15.284934,Cowboys,29,12.225792,6.059142,-4.059142,16.476631
1,1,Falcons,6,-13.712941,Eagles,32,6.004854,-16.717795,-9.282205,86.159338
2,1,Bills,16,3.768246,Steelers,23,-2.309941,9.078187,-16.078187,258.508110
3,1,Panthers,19,-8.666293,Jets,14,-13.919407,8.253115,-3.253115,10.582757
4,1,Bengals,27,-2.456741,Vikings,24,-0.009664,0.552923,2.447077,5.988185


In [49]:
sorted(home_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Buccaneers', 15.284934043953063),
 ('Cowboys', 11.783101256752655),
 ('Patriots', 10.705523413428608),
 ('Packers', 9.381102641726198),
 ('Chiefs', 8.055589165759189),
 ('Titans', 7.403371071257156),
 ('Bills', 3.768246368622654),
 ('Chargers', 3.766466113143305),
 ('Rams', 2.6857020665116527),
 ('49ers', 2.511875328909229),
 ('Ravens', 2.108389741731314),
 ('Seahawks', 1.4580687251840934),
 ('Vikings', 0.6410122351070329),
 ('Broncos', -1.1965401382586778),
 ('Dolphins', -1.294340268488216),
 ('Colts', -1.3655712624882672),
 ('Steelers', -1.4553875004877865),
 ('Eagles', -1.5505862091157867),
 ('Browns', -1.6882493175494342),
 ('Saints', -1.8158468101434098),
 ('Bears', -2.075649110611542),
 ('Bengals', -2.456741190810134),
 ('Commanders', -3.3157060124607964),
 ('Cardinals', -3.5681749988239106),
 ('Lions', -5.136506924909355),
 ('Jaguars', -5.377840761539602),
 ('Raiders', -5.629198828836001),
 ('Giants', -6.435673567789978),
 ('Jets', -8.233089467909377),
 ('Panthers', -8.666292

In [50]:
sorted(away_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Bills', 19.382671545842552),
 ('Cardinals', 15.071608234186051),
 ('Cowboys', 12.225792330789407),
 ('Bengals', 9.23569430043403),
 ('Colts', 9.137272341907714),
 ('Chiefs', 7.543630381794547),
 ('Saints', 7.0237261800691755),
 ('Rams', 6.715046584272154),
 ('Eagles', 6.004853582182128),
 ('49ers', 5.635348240495411),
 ('Patriots', 5.303120915602442),
 ('Buccaneers', 3.403104823678109),
 ('Seahawks', 2.7398847492887017),
 ('Titans', 2.3755095862944198),
 ('Browns', 2.2666956588051286),
 ('Broncos', 2.1663278281342215),
 ('Raiders', 0.19292891508005494),
 ('Vikings', -0.009664292664961605),
 ('Packers', -0.8922465776578185),
 ('Dolphins', -1.7660975193428885),
 ('Ravens', -2.08026754383328),
 ('Chargers', -2.289506503949615),
 ('Steelers', -2.3099410157527536),
 ('Panthers', -3.9921369461996727),
 ('Falcons', -4.001335977661009),
 ('Bears', -5.572284006575848),
 ('Commanders', -5.769610753271373),
 ('Giants', -7.2391598806389075),
 ('Lions', -8.057123466637558),
 ('Texans', -10.00194